# Caravan and Camping Club POI Generator 2020

In [1]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [2]:
# imports
import pandas as pd
import simplekml
import folium

In [3]:
df = pd.read_csv("data/Certificated-Sites.csv", header=0)
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                            Address
0  -1.893108  52.830991  "Abbots Bromley - Little Dunstal Farm WS15 3EP...
1  -4.194925  52.225723  "Aberaeron - Tynffynnon Farm SA48 8DH>01545 57...
2  -4.688622  52.841948    "Aberdaron - Bryn Tawel LL53 8LE>01758 760555""
3  -4.733525  52.804101     "Aberdaron - Bwlchgwyn LL53 8BY>07968 354539""
4  -4.708900  52.847187       "Aberdaron - Cae Hic LL53 8LP>07976 973559""
0    "Abbots Bromley - Little Dunstal Farm WS15 3EP...
1    "Aberaeron - Tynffynnon Farm SA48 8DH>01545 57...
2      "Aberdaron - Bryn Tawel LL53 8LE>01758 760555""
3       "Aberdaron - Bwlchgwyn LL53 8BY>07968 354539""
4         "Aberdaron - Cae Hic LL53 8LP>07976 973559""
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [4]:
#df['SiteName'] = df['Address'].str.extract('([a-zA-Z].*(?=>))')

temp = df['Address'].apply(lambda x: pd.Series(str(x).split('>')))
#print(temp)  # check if data is okay, one line had less quotes in it for some reason!!
df["PhoneNumber"]= temp[1]
# Remove extra quotes the camping club add to the end of each line!
df['PhoneNumber'] = df.apply(lambda x: x['PhoneNumber'][:-2], axis=1)

# making separate phone number column from new data frame 
temp_address = temp[0].apply(lambda x: pd.Series(str(x).split(' - ')))
df["Town"] = temp_address[0]
df["SiteName"] = temp_address[1]

df["Town"] = df.apply(lambda x: x['Town'][1:], axis=1)

# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 

df 

,Longitude,Latitude,PhoneNumber,Town,SiteName
0,-1.893108,52.830991,01283 840342,Abbots Bromley,Little Dunstal Farm WS15 3EP
1,-4.194925,52.225723,01545 570251,Aberaeron,Tynffynnon Farm SA48 8DH
2,-4.688622,52.841948,01758 760555,Aberdaron,Bryn Tawel LL53 8LE
3,-4.733525,52.804101,07968 354539,Aberdaron,Bwlchgwyn LL53 8BY
4,-4.708900,52.847187,07976 973559,Aberdaron,Cae Hic LL53 8LP
...,...,...,...,...,...
1306,-1.049209,54.014443,01904 761387,York,The Lodge YO32 9SW
1307,-1.276889,53.918600,01937 833688,York,Wighill Manor LS24 8BT
1308,-1.205900,53.996800,07825 446817,York,Woodview YO26 8JF
1309,-1.711742,53.173372,01629 636822,Youlgreave,Dukes Piece Site DE45 1US


### Now create kml file for use by maps.me

### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>  *
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl>

In [5]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/blu-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    #name = row['SiteName'] + ', ' + row['SiteAddress'] + ', ' + row['PhoneNumber']
    name = row['SiteName']
    address = row['Town'] + ', ' + row['SiteName'] + ', ' + row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
    
kml.save(path="poi/ccc-cs-2020.kml")
map

In [46]:
#!pip list